In [1]:
import requests
import json
import numpy as np
import itertools

from pymatgen.phasediagram.pdanalyzer import PDAnalyzer
from pymatgen.matproj.rest import MPRester 
from pymatgen.phasediagram.pdmaker import PhaseDiagram
from pymatgen.core.periodic_table import Element
from pymatgen.core.composition import Composition


API_key = 'KSHpuwLlQO8scf94' # Get API key from https://materialsproject.org/open
mp = MPRester(API_key)


In [2]:
### Get all stable Ternary systems in the database

data = {
    'criteria': {"nelements" : 3,"e_above_hull" :0}, # 0 energy above hull (stability measure)
    'properties': ['chemsys']
}
r = requests.post('https://materialsproject.org/rest/v2/query',
                 headers={'X-API-KEY': API_key},
                 data={k: json.dumps(v) for k,v in data.iteritems()})
response = r.json() # a dict of all stable ternary systems (need to check for duplicates)

ternary_sys = []
for e in response['response']:
    ternary_sys.append(e['chemsys'].split('-'))
# remove duplicates
ternary_sys.sort()
ternary_sys = list(ternary_sys for ternary_sys,_ in itertools.groupby(ternary_sys)) 

for i in range(0,len(ternary_sys)):
    ternary_sys[i] = '-'.join(ternary_sys[i])
len(ternary_sys)

8573

In [3]:
# Dict of all Binary systems
data = {
    'criteria': {"nelements" : 1},
    'properties': ['elements']
}
r = requests.post('https://materialsproject.org/rest/v2/query',
                 headers={'X-API-KEY': API_key},
                 data={k: json.dumps(v) for k,v in data.iteritems()})
elements = r.json()

# Create a list of all binary systems.
import itertools
element_list = []
binary_sys = []
for i in range(0 , len (elements['response'])):
    element_list.append(elements['response'][i]['elements'][0])
element_list = list(set(element_list))
for subset in itertools.combinations(element_list,2):
     binary_sys.append(subset[0]+'-'+subset[1])
binary_sys = sorted (binary_sys)    
len(binary_sys) # 3916 systems; combination of 89 known elements

3916

In [4]:
def prop():
    p = np.arange(0.05,1,0.05)
    p = np.around(p,decimals=2)
    p = p.tolist()
    return p

In [5]:
%doctest_mode
def material_load(d, sep='-', p = prop()):
    return_data = []
     
    d = d.split('-')
    
    # Create a phase diagram object for the following system:
    entry = mp.get_entries_in_chemsys([d[0],d[1]]) # gets the entries of the chemical system
    pd = PhaseDiagram(entry) # creates a phasediagram object
    pd_analyse = PDAnalyzer(pd) # creates a phase Diagram analysis object

    # Get the features for various proportions Using the get_hull_energy method;
    # (Need to add documentation)
    for i in range(0,len(p)):
        temp_data = {}
        prop_a = p[i]
        prop_b = p[-(i+1)]
        a = Element(d[0])
        b = Element(d[1])
        try :
            temp_data['system'] = d[0]+'-'+d[1]
            temp_data['A'] = d[0]
            temp_data['B'] = d[1]
            temp_data[d[0]+'_prop'] = prop_a
            temp_data[d[1]+'_prop'] = prop_b
            temp_data['formation_energy'] = pd_analyse.get_hull_energy(Composition.from_dict({d[0]: prop_a, d[1] : prop_b}))

            # Element Property extraction

            temp_data['avg_atomic_mass'] = prop_a*a.atomic_mass + prop_b*b.atomic_mass
            temp_data['avg_row'] = prop_a*a.row + prop_b*b.row
            temp_data['max_z_diff'] = abs (a.Z - b.Z) # Max Difference in atomic number
            temp_data['avg_z'] = prop_a*a.Z + prop_b*b.Z
            temp_data['max_radius_diff'] = abs (a.atomic_radius - b.atomic_radius) # Max Difference in atomic radius
            temp_data['avg_radius'] = prop_a*a.atomic_radius + prop_b*b.atomic_radius
            temp_data['max_en_diff'] = abs (a.X - b.X) # Max Difference in electronegativity
            temp_data['avg_en'] = prop_a*a.X + prop_b*b.X # Avg Difference in electronegativity
            return_data.append(temp_data)
        except TypeError:
            pass
    return return_data

Exception reporting mode: Plain
Doctest mode is: ON


In [18]:
%doctest_mode

import pandas as pd

master = pd.DataFrame()

for i,item in enumerate(binary_sys, 1):
    item = item.encode('ascii')
    temp = pd.DataFrame(material_load(item))
    master.append(temp)
    if i % 10 == 0:
        print i
master

Exception reporting mode: Plain
Doctest mode is: ON
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2

Empty DataFrame
Columns: []
Index: []

Help on class enumerate in module __builtin__:

class enumerate(object)
 |  enumerate(iterable[, start]) -> iterator for index, value of iterable
 |  
 |  Return an enumerate object.  iterable must be another object that supports
 |  iteration.  The enumerate object yields pairs containing a count (from
 |  start, which defaults to zero) and a value yielded by the iterable argument.
 |  enumerate is useful for obtaining an indexed list:
 |      (0, seq[0]), (1, seq[1]), (2, seq[2]), ...
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(...)
 |      x.__getattribute__('name') <==> x.name
 |  
 |  __iter__(...)
 |      x.__iter__() <==> iter(x)
 |  
 |  next(...)
 |      x.next() -> the next value, or raise StopIteration
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __new__ = <built-in method __new__ of type object>
 |      T.__new__(S, ...) -> a new object with type S, a subtype of T



In [ ]:
## Scratch work 
%doctest_mode 
k = Element('Fe')
k.common_oxidation_states
entry = mp.get_entries_in_chemsys(['Ag','As'])
pd = PhaseDiagram(entry)
pd_analyse = PDAnalyzer(pd)
pd_analyse.get_hull_energy(Composition.from_dict({'Ag': 0.02, 'As': 0.98}))
binary_sys[10]
Composition.from_dict({'Fe': 0.02, 'O': 0.98})
i = 1
prop()[-i]

In [18]:
k = pd.DataFrame(material_load('Fe-F'))
m = pd.DataFrame(material_load('Fe-O'))
o = pd.DataFrame(material_load('Na-F'))
m.append([k,o]).max_en_diff

0     1.61
1     1.61
2     1.61
3     1.61
4     1.61
5     1.61
6     1.61
7     1.61
8     1.61
9     1.61
10    1.61
11    1.61
12    1.61
13    1.61
14    1.61
15    1.61
16    1.61
17    1.61
18    1.61
0     2.15
1     2.15
2     2.15
3     2.15
4     2.15
5     2.15
6     2.15
7     2.15
8     2.15
9     2.15
10    2.15
11    2.15
12    2.15
13    2.15
14    2.15
15    2.15
16    2.15
17    2.15
18    2.15
0     3.05
1     3.05
2     3.05
3     3.05
4     3.05
5     3.05
6     3.05
7     3.05
8     3.05
9     3.05
10    3.05
11    3.05
12    3.05
13    3.05
14    3.05
15    3.05
16    3.05
17    3.05
18    3.05
Name: max_en_diff, Length: 57, dtype: float64

In [21]:
from ipy_progressbar import ProgressBar
k = 0
pb = ProgressBar(10, title='Outer', key='outer')
for i in pb:
    pb_inner = ProgressBar(5, title='Inner', key='inner')
    for j in pb_inner:
        k = k+2

SyntaxError: invalid syntax (terminal_bar.py, line 24)

In [16]:
2/float(len(binary_sys))

0.0005107252298263534